In [2]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [3]:
# Generate some parameters. These can be reused.
params = dh.generate_parameters(generator=2, key_size=2048)

In [6]:
type(params)

cryptography.hazmat.backends.openssl.dh._DHParameters

In [21]:
# Generator for DH group.
g = params.parameter_numbers().g
print(f"g = {g}")

g = 2


In [22]:
# Prime modulus value for DH group.
p = params.parameter_numbers().p
print(f"p = {p}")

p = 31527210969358136047390113101184180902894055266460392637339033513316284548774557526785767529208554487442501199476437489250980412191667815035335556942213245682049688333236907312208944565310171013551630724107880506755560439061250314402904080972139905438449160395864186593582346778870653575674392216118251468281216909931478642668550193030705082594652051346111773011385798068044399242209442464166810007368888485800344656588053719274314658041301195842901236444942830160443195101346856775914684966406052350807965237105553128129772591165005425651157173775149774694104320853478311250046609751167848647704281797035906505839827


In [10]:
# Generate a private key for use in the exchange.
server_private_key = params.generate_private_key()

In [23]:
# Private value "x". This is Alice's private "a" value.
a = server_private_key.private_numbers().x
print(f"a = {a}")

a = 9336790721610775047083129078809648819684079204484753481924332939390994831049911792020163832984342227976000372252797347869886772285189796568129889858757386818839643017883916339259377973091620002656388186493775899330538990682626428534846230692813932959801356147180844506322672212791545675518785282853155254787776855229660613458420380759816643297621660756586009675212793293963374885786125302987891630461602410515075264255053198953558786188449060639678033691872909109289335144143501576901155786081751368034576231987341281100775613393445405003179748743129521387681118715049195835025544951823129043097660489905254640280480


In [13]:
# In a real handshake the peer is a remote client. For this example we'll generate another local private key though.
# Note that in a DH handshake both peers must agree on a common set of parameters.
peer_private_key = params.generate_private_key()

In [24]:
# This is Bob's private "b" value.
b = peer_private_key.private_numbers().x
print(f"b = {b}")

b = 12328971636675774495524507270266035894991968143308694155387442919797184223108289344096149887625751922198055104977549795666219137890001701415456519688129386990624812194216976531724152631622581263103156983583417894384236950175372871513820749981141454954602735601111878857729223910435878926474694940662324547006541610144509741439722939923673396121068450971644517128267342958119538703599975197872483495454857286208979187223558938005049952982999243984237211858916420336761258821183170241315312477156310461896627127132166503762278850941184591147978527530273897762659584487403372600506764615495795833535383355060988289866168


In [14]:
shared_key = server_private_key.exchange(peer_private_key.public_key())

 See https://cryptography.io/en/latest/hazmat/primitives/asymmetric/dh/#cryptography.hazmat.primitives.asymmetric.dh.DHPrivateKey.exchange for further detail.

In [19]:
# Note, the bytes are ordered in big endian.
type(shared_key)

bytes

In [26]:
sk = int.from_bytes(shared_key, byteorder="big")

In [27]:
# Confirm shared key has value g^ab.
sk == pow(g, a*b, mod=p)

True

See https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#hkdf.

In [28]:
# Perform key derivation.
derived_key = HKDF(
    algorithm = hashes.SHA256(),
    length = 32,
    salt=None,
    info=b'handshake data'
).derive(shared_key)

In [29]:
# Now we can demonstrate that the handshake performed in the opposite direction gives the same final value.
same_shared_key = peer_private_key.exchange(server_private_key.public_key())

In [30]:
same_derived_key = HKDF(
    algorithm = hashes.SHA256(),
    length = 32,
    salt=None,
    info=b'handshake data'
).derive(same_shared_key)

In [31]:
derived_key == same_derived_key

True

In [32]:
from binascii import hexlify as hexa

In [33]:
print(hexa(derived_key))

b'19316eb80b397ce0ed6798d3175eef8968e9065f0a7f7dbe5fb0925b69b2ded3'


In [34]:
print(hexa(shared_key))

b'3cc9fe51edc7b4f5b9b7f5ee0f7f6c0d92cb679a955d26c2dbf87852e36885f93fbd70f341e5ff5d618d059da7b58006e6a009b831aaaaa6febde8b77678999875ee46570d9f9df4b0e94f7500a877362afdaecb35a692afc59c797d9828557c354e00376414cf1564b0edc4be4df3156123ad0955a8d94695661ef20fa54292335037aad1a6fffcc793101849ac76467544f128a50588264e9438e187c2deac9fb4eab4fa148e234c667b9d71d94ce4df630c39bb01657732c514ca9618d78adec0fbfc6ee707bde8f461515b73d30596ea594039593c2ea6ec50594631ac4360426eac9525002c57adba352e114025d676dbb39fd82015cb5384f9cc6c251e'
